In [ ]:
### market clustering

# %time df = read_market_daily(result_path)
%time mdf = pd.read_parquet('data/mdf.parquet')

%time npdf, sharpe_ratios = mc.normalize_market_df(mdf)

# %time npdf_resid = colwise_linreg(npdf)
%time npdf_resid = pd.read_parquet('data/npdf_resid.parquet')

# %time par_cor_mat = npdf_resid.corr()
%time resid_par_cor_mat = pd.read_parquet('data/partial_cor_mat.parquet')

%time details_df = pd.read_parquet('data/details_df.parquet')

# cluster_lables = cluster_sim_matrix(similarity=resid_par_cor_mat.abs())
%time cluster_lables = pd.read_parquet('data/cluster_lables.parquet')

# symbol_meta = join_symbol_data(details_df, cluster_lables, sharpe_ratios, mdf)
%time symbol_meta = pd.read_parquet('data/symbol_meta.parquet')

# cluster_coheasion = mc.get_cluster_coheasion(sim_df=resid_par_cor_mat.abs(), symbol_meta=symbol_meta, cluster_col='cluster_n200')

In [ ]:
symbol_meta['liquidy_rank'] = symbol_meta.groupby('cluster_n200')['daily_avg_dollar_volume'].rank(method='first')

symbol_meta = symbol_meta.sort_values(['cluster_n200', 'liquidy_rank']).reset_index()

by_cluster = symbol_meta.groupby('cluster_n200')
result = []
for cluster, frame in by_cluster:
    counts = dict(frame['sector'].value_counts())
    counts.pop('', None)  # drop symbols w/o sector/industry info
    result.append(counts)

In [ ]:
cluster_sec = pd.DataFrame(result)    
cluster_sec = cluster_sec.drop(columns=['Finance'])
cluster_sec_pct = cluster_sec.apply(lambda x: 100 * x / float(x.sum()), axis=1)

cluster_sec_pct.style.background_gradient(cmap='coolwarm', axis=0)

# n += 1
n = 61
print(n)
symbol_meta[symbol_meta.cluster_n200==n].sector.value_counts()

sym_clust = symbol_meta[symbol_meta.cluster_n200==n]
sym_clust